In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import os
import sys
import math
import shutil
import subprocess
import numpy as np
import geopandas as gpd
import rasterio
from rasterio.features import geometry_mask
from shapely.ops import unary_union


In [ ]:
# ---------------------------------------------------------------------
# USER INPUTS
# ---------------------------------------------------------------------
DEM_PATH = "/home/bgd253/MTI/ANUGA_Flood/data/DEM_MTI_PART.tif"
RIVER_LINE_PATH = "/home/bgd253/MTI/ANUGA_Flood/data/River_SmoothPart.shp"

# Outputs
BUFFER_POLY_PATH = "/home/bgd253/MTI/ANUGA_Flood/data/river_buffer_poly.shp"
RANDOM_RASTER_PATH = "/home/bgd253/MTI/ANUGA_Flood/data/river_randomfield.tif"

# USER INPUTS
# ---------------------------------------------------------------------
#DEM_PATH = "/mnt/c/Users/bgd253/MTI/GIS_Files_OnePlace/Terrain_Final.dem_rev1.tif"
#RIVER_LINE_PATH = "/mnt/c/Users/bgd253/MTI/GIS_Files_OnePlace/River_StLazare_Smooth.shp"

# Outputs
#BUFFER_POLY_PATH = "/mnt/c/Users/bgd253/MTI/GIS_Files_OnePlace/river_buffer_poly.shp"
#RANDOM_RASTER_PATH = "/mnt/c/Users/bgd253/MTI/GIS_Files_OnePlace/river_randomfield.tif"

# Buffer distance (same CRS units as river shapefile; here metres)
BUFFER_DIST = 30.0

# Random noise parameters
NOISE_MIN = 0.0
NOISE_MAX = 1.0
NODATA_VALUE = -9999.0
RANDOM_SEED = 42
# Outside-buffer value (0 = no refinement interest)
OUTSIDE_VALUE = 0.0


In [ ]:
# ------------------------------
# Mesher config (match mesh_RandomNoise.py)
# ------------------------------
MESHER_CONFIG_PY = "mesher_config_from_notebook.py"

dem_filename   = DEM_PATH
errormetric    = "rmse"
max_tolerance  = 1000.0
min_area       = 100.0
max_area       = 20000.0

simplify       = True
simplify_tol   = 10.0     # IMPORTANT: match your script
lloyd_itr      = 2
MPI_nworkers   = 1
user_output_dir = "./mesh_out"

# Refinement random field
randomfield_filename = RANDOM_RASTER_PATH
randomfield_method   = "mean"
randomfield_tol      = 0.1


In [ ]:
# ---------------------------------------------------------------------
# 1) Build random noise raster inside river buffer (outside = NoData)
# ---------------------------------------------------------------------
def build_randomfield_river_buffer(
    dem_path: str,
    river_path: str,
    buffer_dist: float,
    out_randomfield_path: str,
    noise_min: float = 0.0,
    noise_max: float = 1.0,
    nodata_value: float = -9999.0,
    random_seed: int = 42,
    save_buffer_polygon_shp: str = None,
):
    """
    Create a random-field raster aligned to DEM:
      - Inside river buffer: uniform noise [noise_min, noise_max]
      - Outside buffer: nodata_value

    Output is DEM-aligned (same width/height/transform), so Mesher can sample it reliably.
    """
    rng = np.random.default_rng(random_seed)

    # Read river lines
    rivers = gpd.read_file(river_path)
    if rivers.empty:
        raise RuntimeError(f"River line file appears empty: {river_path}")
    if rivers.crs is None:
        raise RuntimeError("River shapefile has no CRS. Define CRS in GIS first.")
    print(f"[RandomField] River CRS: {rivers.crs}")

    # Buffer and dissolve
    rivers["geometry"] = rivers.geometry.buffer(buffer_dist)
    buffer_geom = unary_union(rivers.geometry)
    buffer_gdf = gpd.GeoDataFrame({"id": [1]}, geometry=[buffer_geom], crs=rivers.crs)

    if save_buffer_polygon_shp:
        buffer_gdf.to_file(save_buffer_polygon_shp)
        print(f"[RandomField] Saved buffer polygon: {save_buffer_polygon_shp}")

    # Open DEM template
    with rasterio.open(dem_path) as dem:
        profile   = dem.profile.copy()
        width     = dem.width
        height    = dem.height
        transform = dem.transform
        dem_crs   = dem.crs

    print(f"[RandomField] DEM size: {width} x {height}, CRS: {dem_crs}")

    # Reproject buffer to DEM CRS if needed
    if buffer_gdf.crs != dem_crs:
        print("[RandomField] Reprojecting buffer to DEM CRS...")
        buffer_gdf = buffer_gdf.to_crs(dem_crs)

    # Mask: True inside polygon, False outside
    inside_mask = geometry_mask(
        [buffer_gdf.geometry.iloc[0]],
        transform=transform,
        invert=True,               # True inside polygon
        out_shape=(height, width),
    )

    # Noise everywhere, then set outside to nodata
    noise = rng.uniform(noise_min, noise_max, size=(height, width)).astype("float32")
    noise_masked = np.where(inside_mask, noise, nodata_value).astype("float32")

    # Write raster
    profile.update(
        dtype="float32",
        nodata=nodata_value,
        count=1,
        compress="lzw",
    )

    out_dir = os.path.dirname(out_randomfield_path)
    if out_dir and not os.path.exists(out_dir):
        os.makedirs(out_dir, exist_ok=True)

    with rasterio.open(out_randomfield_path, "w", **profile) as dst:
        dst.write(noise_masked, 1)

    print(f"[RandomField] Saved random field raster: {out_randomfield_path}")




In [ ]:
# ---------------------------------------------------------------------
# 2) Write Mesher config that Mesher actually uses (parameter_files)
# ---------------------------------------------------------------------
def write_mesher_config(
    config_path: str,
    dem_filename: str,
    errormetric: str,
    max_tolerance: float,
    min_area: float,
    max_area: float,
    simplify: bool,
    simplify_tol: float,
    lloyd_itr: int,
    MPI_nworkers: int,
    user_output_dir: str,
    parameter_files: dict | None = None,
):
    """
    Write a Mesher config *.py file using ONLY passed arguments (no notebook globals).
    This avoids NameError issues in notebooks and makes runs reproducible.
    """

    parameter_files = parameter_files or {}

    # Pull random field settings safely
    rf = parameter_files.get("random_field", {})
    rf_file = rf.get("file", None)
    rf_method = rf.get("method", "mean")
    rf_tol = rf.get("tolerance", 0.1)

    # Only include random_field block if a file is provided (and non-empty)
    if rf_file:
        rf_block = f'''
parameter_files = {{
  "random_field": {{
    "file": r"{rf_file}",
    "method": "{rf_method}",
    "tolerance": {rf_tol}
  }}
}}
'''
    else:
        rf_block = "parameter_files = {}\n"

    # For WSL debugging, force single worker unless you *know* MPI is stable.
    try:
        MPI_nworkers = max(1, int(MPI_nworkers))
    except Exception:
        MPI_nworkers = 1

    config_text = f'''#!/usr/bin/env python
# Auto-generated Mesher config (from notebook)

dem_filename    = r"{dem_filename}"
errormetric     = "{errormetric}"
max_tolerance   = {max_tolerance}
min_area        = {min_area}
max_area        = {max_area}

{rf_block}

simplify        = {simplify}
simplify_tol    = {simplify_tol}

lloyd_itr       = {lloyd_itr}
MPI_nworkers    = {MPI_nworkers}
user_output_dir = r"{user_output_dir}"

if __name__ == "__main__":
    # config module only; Mesher runner imports this file
    pass
'''

    with open(config_path, "w", encoding="utf-8") as f:
        f.write(config_text)

    print(f"[MesherConfig] Wrote config: {os.path.abspath(config_path)}")


In [ ]:
# ---------------------------------------------------------------------
# 3) Run Mesher from config (robust: tries `mesher` then `python -m mesher`)
# ---------------------------------------------------------------------
def run_mesher_from_config(config_path: str, output_dir: str, parameter_files: dict | None = None):
    print("\n==============================")
    print("MESHER RUN DIAGNOSTICS")
    print("==============================")
    print("Python :", sys.executable)
    print("CWD    :", os.getcwd())
    print("Config :", os.path.abspath(config_path))

    if not os.path.exists(config_path):
        raise RuntimeError(f"Config not found: {config_path}")

    out_abs = os.path.abspath(output_dir)
    os.makedirs(out_abs, exist_ok=True)
    print("Output dir:", out_abs)

    # Optional sanity check: random field exists (prevents running a no-refinement mesh by mistake)
    parameter_files = parameter_files or {}
    rf = parameter_files.get("random_field", {})
    rf_file = rf.get("file", None)
    if rf_file:
        print("Random field:", rf_file, "exists:", os.path.exists(rf_file))
        if not os.path.exists(rf_file):
            raise RuntimeError(f"Random field raster not found: {rf_file}")

    mesher_exe = shutil.which("mesher")
    print("which mesher:", mesher_exe)

    candidates = []
    if mesher_exe:
        candidates.append([mesher_exe, os.path.abspath(config_path)])
    candidates.append([sys.executable, "-m", "mesher", os.path.abspath(config_path)])

    last_err = None
    for cmd in candidates:
        print("\nTrying:\n  " + " ".join(cmd))
        try:
            subprocess.run(cmd, check=True)
            print("\n✅ Mesher finished successfully.")
            print("Check outputs in:", out_abs)
            return
        except Exception as e:
            print("❌ Failed:", repr(e))
            last_err = e

    raise RuntimeError(
        "Mesher could not be executed.\n"
        "Either Mesher isn't installed in this environment or the command differs.\n"
        f"Last error: {last_err}"
    )


# ---------------------------------------------------------------------
# MAIN (for .py execution)
# NOTE: In Jupyter, DO NOT rely on this block.
#       Run the three calls manually in cells (same order).
# ---------------------------------------------------------------------
def main():
    # 1) Build random field raster (river buffer noise)
    build_randomfield_river_buffer(
        dem_path=DEM_PATH,
        river_path=RIVER_LINE_PATH,
        buffer_dist=BUFFER_DIST,
        out_randomfield_path=RANDOM_RASTER_PATH,
        noise_min=NOISE_MIN,
        noise_max=NOISE_MAX,
        nodata_value=NODATA_VALUE,
        random_seed=RANDOM_SEED,
    )

    # IMPORTANT: define parameter_files explicitly (no notebook globals inside helpers)
    parameter_files = {
        "random_field": {
            "file": RANDOM_RASTER_PATH,
            "method": "mean",
            "tolerance": 0.1,
        }
    }

    # 2) Write config Mesher understands
    write_mesher_config(
        config_path=MESHER_CONFIG_PY,
        dem_filename=dem_filename,
        errormetric=errormetric,
        max_tolerance=max_tolerance,
        min_area=min_area,
        max_area=max_area,
        simplify=simplify,
        simplify_tol=simplify_tol,
        lloyd_itr=lloyd_itr,
        MPI_nworkers=MPI_nworkers,
        user_output_dir=user_output_dir,
        parameter_files=parameter_files,
    )

    # Optional: print config for sanity
    print("\n----- CONFIG CONTENT -----")
    print(open(MESHER_CONFIG_PY, "r", encoding="utf-8").read())
    print("--------------------------\n")

    # 3) Run Mesher
    run_mesher_from_config(MESHER_CONFIG_PY, user_output_dir, parameter_files=parameter_files)


if __name__ == "__main__":
    main()
